# $$\text{ARTIST NETWORK}$$

In [235]:
import spotipy
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
from pyvis.network import Network

%matplotlib inline

## Network of top artists
The list of top artists that a user listened to are compiled into a network
- Each artist seem to have 30 related artists (edges)
- The total number of edges in the network is roughly 1000
- You might be able to see the artist that your listening is centered around

In [236]:
username = '2273exo4mylzm3zcnbcgw76xa'
scope = 'user-library-read user-top-read user-follow-read app-remote-control streaming'
client_id = 'b2e1883e5add45d4bb51814bf69fc387'
client_secret = 'a834408986044bba8a41539f40dcef47'
redict_uri = 'http://localhost:8888/callback'

import requests
from spotipy import util

In [237]:
tk = util.prompt_for_user_token(username, scope, client_id, client_secret, redict_uri)

In [238]:
sp = spotipy.Spotify(auth=tk)

In [239]:
auth = {"Authorization": "Bearer {}".format(tk)}

SPOTIFY_API_BASE_URL = "https://api.spotify.com"
API_VERSION = "v1"
SPOTIFY_API_URL = "{}/{}".format(SPOTIFY_API_BASE_URL, API_VERSION)
USER_PROFILE_ENDPOINT = "{}/{}".format(SPOTIFY_API_URL, 'me')
USER_TOP_ARTISTS_ENDPOINT = "{}/{}".format(USER_PROFILE_ENDPOINT, 'top')

def user_top_artists(auth_header, time_range, limit):
    url = '{}/top/artists?time_range={}&limit={}'.format(USER_PROFILE_ENDPOINT, time_range, limit)
    resp = requests.get(url, headers=auth_header)
    return resp.json()

def artist_related_artists(auth_header, artist_id):
    url = '{}/artists/{}/related-artists'.format(SPOTIFY_API_URL, artist_id)
    resp = requests.get(url, headers=auth_header)
    return resp.json()

artists_long = pd.json_normalize(user_top_artists(auth,'long_term', 30)['items'])
artists_medium = pd.json_normalize(user_top_artists(auth,'medium_term', 30)['items'])
artists_short = pd.json_normalize(user_top_artists(auth,'short_term', 30)['items'])

artists_fusion = pd.concat([artists_long, artists_medium, artists_short])
network_lists, edges = [artists_fusion], []

In [240]:
for i in range(len(artists_fusion)):
    temp = []
    related_artists = pd.json_normalize(artist_related_artists(auth, artists_fusion.iloc[i]['id'])['artists'])[:20]
    for j in range(len(related_artists)):
        if related_artists.iloc[j]['name'].isnumeric(): continue
        else: temp.append((artists_fusion.iloc[i]['name'], related_artists.iloc[j]['name']))
    network_lists.append(related_artists)
    edges.append(temp)
    
network = pd.concat(network_lists).drop_duplicates(subset=['id']).reset_index(drop=True)
network = network.sort_values(by=['name'])

In [177]:
network = network[network.name.isin([name for name in network.name if not name.isnumeric()])]
network = network.drop_duplicates(subset=['name'])
network.reset_index(drop=True, inplace=True)

In [194]:
images = []
for i in range (len(network)):
    try: images.append(network.iloc[i]['images'][2]['url'])
    except: images.append(None)

In [251]:
pd.DataFrame(graph_edges).to_csv('edges.csv', index=False)

In [215]:
graph_edges = sum(edges, [])
graph = nx.Graph()
graph.add_edges_from(graph_edges)
size = [i[1] for i in sorted(dict(graph.degree).items())]
g = Network(height="100%", width="100%", notebook=False)
g.inherit_edge_colors(True)
nodes = [i[0] for i in sorted(dict(graph.degree).items())] 
g.barnes_hut()

for idx,i in enumerate(nodes):
    g.add_node(i, value=size[idx], label=' ', title=i, shape='circularImage', image=images[idx])
    
g.add_edges(graph.edges)
#g.show_buttons(filter_=True)

g.set_options('''
var options = {
  "nodes": {
    "shape": "diamond",
    "shadow": {
      "enabled": true
    },
    "size": 200
  },
  "color": {
      "highlight": {
        "border": "rgba(233,33,28,1)",
        "background": "rgba(255,94,100,1)"
      }
    },
  "edges": {
    "color": {
      "highlight": "rgba(216,83,84,1)",
      "inherit": true,
      "opacity": 0.25
    },
    "shadow": {
      "enabled": true
    },
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    }
  },
  "physics": {
    "forceAtlas2Based": {
      "springLength": 100
    },
    "minVelocity": 0.75,
    "solver": "forceAtlas2Based"
  }
}
''')

In [234]:
g.show('network.html')

In [233]:
ranking = pd.DataFrame([i for i in sorted(dict(graph.degree).items())], columns=['Artist', 'Connections'])
ranking[~ranking['Artist'].isin(list(set(artists_fusion.name)))].sort_values(by='Connections', ascending=False).head(20)

,Artist,Connections
536,Nass El Ghiwane,6
600,Raïna Raï,5
590,Quavo,5
670,Soprano,4
493,Medine,4
433,Lil Uzi Vert,4
279,Gradur,4
87,Bigflo & Oli,4
781,Young Thug,4
240,Faudel,4


# Other section

In [ ]:
'''
Remarks:
The higher the degree of an artists the most likely that a user would like their style of music
The idea is to cross over the artists based on the top artists that the user listened to the most


Next step is to differentiate between the artists already in the top 50 and the once that are shared
as similar to one or more artists from the top 50
Most likely, there are going to be artists that the user know about but didn't make it to top 50 
I guess we're going to bank on the idea that the user didn't explore them yet
Or who cares! the gist is just to leave them for people to see.
''';